In [1]:
'''Basic package'''
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import queue
import cv2          #影像處理
import scipy.misc   #影像處理
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm #進度條
import matplotlib.pyplot as plt #繪圖
import datetime
import xgboost
import sklearn
import matplotlib.pyplot as plt
% pylab inline

import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping, ModelCheckpoint



Populating the interactive namespace from numpy and matplotlib


Using TensorFlow backend.


In [2]:
#抓入之前產出的2張表，還有training-set.csv和testing-set.csv
file_stat = pd.read_csv('total_data_analysis.csv')
#把index欄位drop掉
label =pd.read_csv('/data/examples/trend/data/training-set.csv',
                   names =('FileID','label'),
                   header=None)
pivot_table_prod = pd.read_csv('pivot_table_prod.csv')
pivot_table_date = pd.read_csv('pivot_table_date.csv')
pivot_table_hour = pd.read_csv('pivot_table_hour.csv')
pivot_table_week = pd.read_csv('pivot_table_week.csv')

submission = pd.read_csv('/data/examples/trend/data/testing-set.csv',
                   names =('FileID','label'),
                   header=None)
#分別把testing跟training的label跟兩張表merge在一起
pivot_merge = pd.merge(pd.merge(pd.merge(pivot_table_prod,pivot_table_date,how='inner',on='FileID'),
                                pivot_table_hour,how='inner',on='FileID'),pivot_table_week,how='inner',on='FileID')
testing_data= pd.merge(submission,pd.merge(file_stat,pivot_merge,how='inner',on='FileID'),how='inner',on='FileID')
merge = pd.merge(pd.merge(file_stat,pivot_merge,how='inner',on='FileID'),label,how='inner',on='FileID')

In [3]:
#因為病毒資料數較少，為了避免直接做train_test_split會分布不均，所以各自病毒跟正常的FileID分開
malware = merge.loc[merge['label']==1]
normal = merge.loc[merge['label']==0]
print(malware.shape,normal.shape)
#針對DF作取樣20%
malware_test = malware.sample(frac=0.2)
#把取樣取出的樣本從訓練及拿掉
malware_train = malware.drop(malware_test.iloc[:,0:0].index.tolist())
#針對DF作取樣20%
normal_test = normal.sample(frac=0.2)
#把取樣取出的樣本從訓練及拿掉
normal_train = normal.drop(normal_test.iloc[:,0:0].index.tolist())

print(malware_test.shape,normal_test.shape)
print(malware_train.shape,normal_train.shape)


(5640, 164) (46878, 164)
(1128, 164) (9376, 164)
(4512, 164) (37502, 164)


In [4]:
#把病毒跟正常檔案合併回來
df_train = normal_train.append(malware_train).sample(frac=1)
df_test = normal_test.append(malware_test).sample(frac=1)

In [5]:
df_train.shape,df_test.shape

((42014, 164), (10504, 164))

In [6]:
#把最後一個column：label去除、x_train跟x_test還要把FileID拿掉
x_train = df_train.iloc[:,1:-1]

#feature_name抓出來，理論上在XGBoost的時候可以把column name抓回去畫圖，但還不會做
feature_name = x_train.columns

x_test = df_test.iloc[:,1:-1]
y_train = df_train.iloc[:,-1]
y_test = df_test.iloc[:,-1]

#轉成array（因為training好像是吃array）
x_train = x_train.as_matrix()
x_test = x_test.as_matrix()
y_train = y_train.as_matrix()
y_test = y_test.as_matrix()




In [7]:
shape(x_train),shape(y_train),shape(x_test),shape(y_test)

((42014, 162), (42014,), (10504, 162), (10504,))

### DNN

In [8]:
num_classes = 2
epochs = 30
model_name = 'trendmicro'

# Convert class vectors to binary class matrices.
# y_train = keras.utils.to_categorical(y_train, num_classes)
# y_test = keras.utils.to_categorical(y_test, num_classes)

model_dnn = Sequential()
model_dnn.add(Dense(128, input_dim=x_train.shape[1], activation='relu'))
model_dnn.add(Dense(256))
model_dnn.add(Activation('sigmoid'))
model_dnn.add(Dropout(0.3))
model_dnn.add(Dense(512))
model_dnn.add(Activation('sigmoid'))
model_dnn.add(Dropout(0.3))
model_dnn.add(Dense(1))
model_dnn.add(Activation('sigmoid'))

print(model_dnn.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               20864     
_________________________________________________________________
dense_2 (Dense)              (None, 256)               33024     
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               131584    
_________________________________________________________________
activation_2 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
__________

In [9]:

# initiate Adagrad optimizer
opt = keras.optimizers.Adagrad()

# Let's train the model using Adam
model_dnn.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['binary_accuracy'])

# earlystop
earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=1)


In [10]:
# Fit the model on the batches generated by datagen.flow().
model_dnn.fit(x_train, y_train,
                    epochs=epochs,
                    validation_data=(x_test, y_test),
)
#                     callbacks=[checkpoint, earlystop])
# loading our save model
print("Loading trained model")
# model = load_model(model_path)

# Score trained model.
scores = model_dnn.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])


Train on 42014 samples, validate on 10504 samples
Epoch 1/30
42014/42014 [==============================] - 22s 535us/step - loss: 0.2789 - binary_accuracy: 0.8944 - val_loss: 0.2548 - val_binary_accuracy: 0.9015
Epoch 2/30
42014/42014 [==============================] - 18s 420us/step - loss: 0.2470 - binary_accuracy: 0.9059 - val_loss: 0.2377 - val_binary_accuracy: 0.9079
Epoch 3/30
42014/42014 [==============================] - 17s 407us/step - loss: 0.2307 - binary_accuracy: 0.9128 - val_loss: 0.2273 - val_binary_accuracy: 0.9132
Epoch 4/30
42014/42014 [==============================] - 13s 307us/step - loss: 0.2209 - binary_accuracy: 0.9180 - val_loss: 0.2216 - val_binary_accuracy: 0.9160
Epoch 5/30
42014/42014 [==============================] - 19s 442us/step - loss: 0.2117 - binary_accuracy: 0.9220 - val_loss: 0.2178 - val_binary_accuracy: 0.9182
Epoch 6/30
42014/42014 [==============================] - 17s 410us/step - loss: 0.2041 - binary_accuracy: 0.9250 - val_loss: 0.2152 - 

In [12]:
y_pred = model_dnn.predict(x_test)
fpr, tpr, thresholds = sklearn.metrics.roc_curve(y_test, y_pred)
sklearn.metrics.auc(fpr,tpr)

0.90414734012296361

In [15]:
#因為submission原本抓進來是按照FileID排序，但在前面merge的時候亂掉，這裡要先用sort把他換回來
testing_data=testing_data.sort_values('FileID')
#去除FileID跟label的欄位
testing_set=testing_data.drop(['FileID','label'],axis=1)
#轉成array（因為training好像是吃array）
testing_set=testing_set.as_matrix()
#predict_proba出來才會是百分比
predict = model_dnn.predict_proba(testing_set)

In [16]:
#predict
submission['Probability']=predict
submission=submission.drop(['label'],axis=1)
submission.columns=['FileID','Probability']
submission.head()

,FileID,Probability
0,00008c73ee43c15b16c26b26398c1577,0.000227
1,0002ded3a0b54f2ffdab0ca77a5ce2b6,0.959523
2,00050a9df8e13f6ab5a3d3b3e2fc6a86,0.000079
3,000b1aa62b95e448784b8b341de46c64,0.007668
4,000d9f96b5eddf04a3b7a37cb95d0a00,0.137049


In [ ]:
#匯出
submission.to_csv('submission.csv',index=False)